In [369]:
from queue import Queue
from collections import OrderedDict
import copy

In [459]:
class SignalException(Exception):
    pass

class FlipFlop():
    def __init__(self, name, targets):
        # print(f'Creating flipflop {name}')
        self.name = name
        self.curstate = False
        self.targets = targets
        self.initialized = False

    def process(self, name, signal, pq):
        # print(f'{self.name} received {signal} from {name}')
        if not signal:
            self.curstate = not self.curstate  # Flip
            # print(f'{self.name} sending {signal}, state is now {self.curstate}')
            for t in self.targets:
                pq.put((t, self.name, self.curstate))

    def state(self):
        return f'{self.name}_{self.curstate}'

    def init(self, name, mdict):
        if not self.initialized:
            for t in self.targets:
                if t in mdict:
                    mdict[t].init(self.name, mdict)
            self.initialized = True

class Conjunction():
    def __init__(self, name, targets):
        # print(f'Creating conj {name}')
        self.name = name
        self.targets = targets
        self.inputs = {}
        self.trigger = False

    def process(self, name, signal, pq):
        # print(f'{self.name} received {signal} from {name}')
        self.inputs[name] = signal

        signal = False
        for v in self.inputs.values():
            if not v:
                signal = True
                break
        # print(f'{self.name} sending {signal}, input is {self.inputs}')

        if self.trigger and not signal:
            raise SignalException()

        for t in self.targets:
            pq.put((t, self.name, signal))

    def state(self):
        return f'{self.name}_{self.inputs}'

    def init(self, name, mdict):
        if name not in self.inputs:
            self.inputs[name] = False
            for t in self.targets:
                if t in mdict:
                    mdict[t].init(self.name, mdict)

class Broadcast():
    def __init__(self, name, targets):
        self.name = name
        self.signal = False
        self.targets = targets

    def process(self, _name, signal, pq):
        for t in self.targets:
            pq.put((t, self.name, signal))

    def state(self):
        return ''

    def init(self, name, mdict):
        for t in self.targets:
            if t in mdict:
                mdict[t].init(self.name, mdict)

class Output():
    def __init__(self, name, targets):
        self.name = name
        self.n_low = 0
        self.n_high = 0

    def process(self, name, signal, pq):
        # print(f'{self.name} received {signal} from {name}')
        if signal:
            self.n_high += 1
        else:
            self.n_low +=1 

    def reset(self):
        self.n_high = 0
        self.n_low = 0

    def state(self):
        return ''

    def init(self, name, mdict):
        return    
        
def push_button(pq):
    pq.put(('broadcaster', 'button', False))

typedict = {'broadcaster': Broadcast, '%': FlipFlop, '&': Conjunction}

In [460]:
def process(mdict, pdict):

    pq = Queue()

    push_button(pq)

    while not pq.empty():
        target, source, signal = pq.get()
        pdict[signal] += 1
        if target in mdict:
            mdict[target].process(source, signal, pq)

    # Done processing, save the state

In [461]:
def loop(mdict):

    pdict={True: 0, False: 0}

    i = 1
    while i<=1000:
        i += 1
        nextstate = process(mdict, pdict)
        # if nextstate in knownstates:
        #     # i -= 1
        #     print('Loop at ', i)
        #     break

    nrep = 1000 / (i-1)

    return nrep * pdict[True] * nrep * pdict[False]

In [462]:
# Needed high: ph, vn, kt, hn
# Input to these must be low

# Input nh -> ph -> nh must send low

# nh, mf, fd and kb must send low signal
# nh 3907 mf 3797 fd 4093 kb 4021
# 
    
def loop_rx(fname):

    presses = []

    for t in ['nh', 'mf', 'fd', 'kb']:
    
        mdict = read_input(fname)
        mdict[t].trigger = True

        pdict={True: 0, False: 0}

        i = 1
        while True:
            try:
                nextstate = process(mdict, pdict)
            except SignalException:
                presses.append(i)
                print(f'{t} triggered at {i}')
                break
            i += 1

    return math.lcm(*presses)

In [463]:
def read_input(fname):
    mdict = OrderedDict()
    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue            
            name, targets = line.strip().split(' -> ')
            targets = targets.split(', ')
            if name == 'broadcaster':
                mdict[name] = Broadcast(name, targets)
            else:
                type = name[0]
                name = name[1:]
                mdict[name] = typedict[type](name, targets)

    mdict['rx'] = Output('rx', [])
    mdict['trigger'] = None
    mdict['broadcaster'].init('button', mdict)
    return mdict

In [466]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

mdict = read_input('input20a.txt')

n_pulses = loop(mdict)
    
print(f'N loops is {n_pulses}')

assert n_pulses == 32000000

print('\nTest case 2\n')

mdict = read_input('input20b.txt')

n_pulses = loop(mdict)
    
print(f'N loops is {n_pulses}')

assert n_pulses == 11687500

print('\nPuzzle case\n')

mdict = read_input('input20.txt')

n_pulses = loop(mdict)
    
print(f'N loops is {n_pulses}')

assert n_pulses == 743871576

print('\n*****\nPuzzle2\n*****\n')

print('Puzzle case\n')

n_pulses = loop_rx('input20.txt')
    
print(f'\nN loops is {n_pulses}')

assert n_pulses == 244151741342687


*****
Puzzle1
*****

Test case

N loops is 32000000.0

Test case 2

N loops is 11687500.0

Puzzle case

N loops is 743871576.0

*****
Puzzle2
*****

Puzzle case

nh triggered at 3907
mf triggered at 3797
fd triggered at 4093
kb triggered at 4021

N loops is 244151741342687
